# PHÂN TÍCH HIỆU SUẤT HỌC TẬP CỦA SINH VIÊN

Thành viên:
- 23C24004 - Lê Nhựt Nam
- 23C24005 - Phạm Thừa Tiểu Thành

## Giới thiệu chung

## Phát biểu bài toán

## Giới thiệu về dữ liệu

## Các câu hỏi phân tích

## Import thư viện

In [2]:
library(dplyr)
library(tidyr)
library(car)
library(readxl)
library(mice)
library(VIM)
library(grid)
library(ggplot2)
library(cowplot)
library(missMDA)
library(FactoMineR)
library(TidyDensity)
library(MASS)
library(leaps)
library(lmtest)
library(Metrics)

## Hàm phụ trợ

In [3]:
remove_high_vif <- function(regr_variable, data, threshold = 5) {      
    stop <- FALSE
    while(stop == FALSE) {
        # Tính VIF cho các biến độc lập
        model <- lm(as.formula(paste(regr_variable, "~ .")), data = data)
        vif_values <- vif(model)
        # Tìm các biến có VIF cao hơn ngưỡng
        high_vif <- which(vif_values > threshold)
        
        if (length(high_vif) > 0) {
            # Loại bỏ các biến có VIF cao
            remove_index <- which.max(high_vif)
            rm_names <- names(data[, !names(data) %in% regr_variable])
            rm_name <- rm_names[remove_index]
            data <- data[, !names(data) %in% rm_name]
        } else {
            stop <- TRUE
        }
    }
    return(data)
}

## Đọc dữ liệu

In [7]:
# Đọc dữ liệu từ tập tin
d1=read.table("../../data/part2/student+performance/student-mat.csv",sep=";",header=TRUE)
d2=read.table("../../data/part2/student+performance/student-por.csv",sep=";",header=TRUE)

raw_data=merge(d1,d2,by=c("school","sex","age","address","famsize","Pstatus","Medu","Fedu","Mjob","Fjob","reason","nursery","internet"))
print(nrow(d3)) # 382 students
str(raw_data)


[1] 382
'data.frame':	382 obs. of  53 variables:
 $ school      : chr  "GP" "GP" "GP" "GP" ...
 $ sex         : chr  "F" "F" "F" "F" ...
 $ age         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ address     : chr  "R" "R" "R" "R" ...
 $ famsize     : chr  "GT3" "GT3" "GT3" "GT3" ...
 $ Pstatus     : chr  "T" "T" "T" "T" ...
 $ Medu        : int  1 1 2 2 3 3 3 2 3 3 ...
 $ Fedu        : int  1 1 2 4 3 4 4 2 1 3 ...
 $ Mjob        : chr  "at_home" "other" "at_home" "services" ...
 $ Fjob        : chr  "other" "other" "other" "health" ...
 $ reason      : chr  "home" "reputation" "reputation" "course" ...
 $ nursery     : chr  "yes" "no" "yes" "yes" ...
 $ internet    : chr  "yes" "yes" "no" "yes" ...
 $ guardian.x  : chr  "mother" "mother" "mother" "mother" ...
 $ traveltime.x: int  2 1 1 1 2 1 2 2 2 1 ...
 $ studytime.x : int  4 2 1 3 3 3 3 2 4 4 ...
 $ failures.x  : int  1 2 0 0 2 0 2 0 0 0 ...
 $ schoolsup.x : chr  "yes" "yes" "yes" "yes" ...
 $ famsup.x    : chr  "yes" "yes" "yes" "

## Khám phá và tiền xử lý dữ liệu

### Dữ liệu có bao nhiều dòng và bao nhiêu cột?

In [8]:
# Kích thước của dữ liệu
# Ta thấy dữ liệu có 231 dòng và 14 cột
dim(raw_data)

[1] 382  53

### Mỗi dòng có ý nghĩa gì? Liệu có tồn tại dòng nào mà mang ý nghĩa khác với các dòng còn lại không?

Dựa trên thông tin của tập dữ liệu, ta thấy mỗi dòng mang ý nghĩa khác nhau, tức là mỗi quan trắc độc lập nhau.

### Dữ liệu có bị trùng lặp không?

In [9]:
# Kiểm tra dữ liệu trùng lặp
duplicates <- raw_data[duplicated(raw_data), ]
duplicate_counts <- table(raw_data[duplicated(raw_data), ])
duplicates # Không có dữ liệu trùng lặp

Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,⋯,famrel.y,freetime.y,goout.y,Dalc.y,Walc.y,health.y,absences.y,G1.y,G2.y,G3.y
<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>


### Mỗi cột mang ý nghĩa gì?

In [10]:
# Đầu tiên, ta xem xét một số quan trắc
str(raw_data)

'data.frame':	382 obs. of  53 variables:
 $ school      : chr  "GP" "GP" "GP" "GP" ...
 $ sex         : chr  "F" "F" "F" "F" ...
 $ age         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ address     : chr  "R" "R" "R" "R" ...
 $ famsize     : chr  "GT3" "GT3" "GT3" "GT3" ...
 $ Pstatus     : chr  "T" "T" "T" "T" ...
 $ Medu        : int  1 1 2 2 3 3 3 2 3 3 ...
 $ Fedu        : int  1 1 2 4 3 4 4 2 1 3 ...
 $ Mjob        : chr  "at_home" "other" "at_home" "services" ...
 $ Fjob        : chr  "other" "other" "other" "health" ...
 $ reason      : chr  "home" "reputation" "reputation" "course" ...
 $ nursery     : chr  "yes" "no" "yes" "yes" ...
 $ internet    : chr  "yes" "yes" "no" "yes" ...
 $ guardian.x  : chr  "mother" "mother" "mother" "mother" ...
 $ traveltime.x: int  2 1 1 1 2 1 2 2 2 1 ...
 $ studytime.x : int  4 2 1 3 3 3 3 2 4 4 ...
 $ failures.x  : int  1 2 0 0 2 0 2 0 0 0 ...
 $ schoolsup.x : chr  "yes" "yes" "yes" "yes" ...
 $ famsup.x    : chr  "yes" "yes" "yes" "yes" ...

| Variable Name | Role    | Type        | Demographic     | Description                                                                                                                                            | Units | Missing Values |
|---------------|---------|-------------|-----------------|--------------------------------------------------------------------------------------------------------------------------------------------------------|-------|----------------|
| school        | Feature | Categorical |                 | student's school (binary: 'GP' - Gabriel Pereira or 'MS' - Mousinho da Silveira)                                                                       |       | no             |
| sex           | Feature | Binary      | Sex             | student's sex (binary: 'F' - female or 'M' - male)                                                                                                     |       | no             |
| age           | Feature | Integer     | Age             | student's age (numeric: from 15 to 22)                                                                                                                 |       | no             |
| address       | Feature | Categorical |                 | student's home address type (binary: 'U' - urban or 'R' - rural)                                                                                       |       | no             |
| famsize       | Feature | Categorical | Other           | family size (binary: 'LE3' - less or equal to 3 or 'GT3' - greater than 3)                                                                             |       | no             |
| Pstatus       | Feature | Categorical | Other           | parent's cohabitation status (binary: 'T' - living together or 'A' - apart)                                                                            |       | no             |
| Medu          | Feature | Integer     | Education Level | mother's education (numeric: 0 - none, 1 - primary education (4th grade), 2 - 5th to 9th grade, 3 - secondary education or 4 - higher education)       |       | no             |
| Fedu          | Feature | Integer     | Education Level | father's education (numeric: 0 - none, 1 - primary education (4th grade), 2 â€“ 5th to 9th grade, 3 â€“ secondary education or 4 â€“ higher education) |       | no             |
| Mjob          | Feature | Categorical | Occupation      | mother's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other')                       |       | no             |
| Fjob          | Feature | Categorical | Occupation      | father's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other')                       |       | no             |
| reason        | Feature | Categorical |                 | reason to choose this school (nominal: close to 'home', school 'reputation', 'course' preference or 'other')                                           |       | no             |
| guardian      | Feature | Categorical |                 | student's guardian (nominal: 'mother', 'father' or 'other')                                                                                            |       | no             |
| traveltime    | Feature | Integer     |                 | home to school travel time (numeric: 1 - <15 min., 2 - 15 to 30 min., 3 - 30 min. to 1 hour, or 4 - >1 hour)                                           |       | no             |
| studytime     | Feature | Integer     |                 | weekly study time (numeric: 1 - <2 hours, 2 - 2 to 5 hours, 3 - 5 to 10 hours, or 4 - >10 hours)                                                       |       | no             |
| failures      | Feature | Integer     |                 | number of past class failures (numeric: n if 1<=n<3, else 4)                                                                                           |       | no             |
| schoolsup     | Feature | Binary      |                 | extra educational support (binary: yes or no)                                                                                                          |       | no             |
| famsup        | Feature | Binary      |                 | family educational support (binary: yes or no)                                                                                                         |       | no             |
| paid          | Feature | Binary      |                 | extra paid classes within the course subject (Math or Portuguese) (binary: yes or no)                                                                  |       | no             |
| activities    | Feature | Binary      |                 | extra-curricular activities (binary: yes or no)                                                                                                        |       | no             |
| nursery       | Feature | Binary      |                 | attended nursery school (binary: yes or no)                                                                                                            |       | no             |
| higher        | Feature | Binary      |                 | wants to take higher education (binary: yes or no)                                                                                                     |       | no             |
| internet      | Feature | Binary      |                 | Internet access at home (binary: yes or no)                                                                                                            |       | no             |
| romantic      | Feature | Binary      |                 | with a romantic relationship (binary: yes or no)                                                                                                       |       | no             |
| famrel        | Feature | Integer     |                 | quality of family relationships (numeric: from 1 - very bad to 5 - excellent)                                                                          |       | no             |
| freetime      | Feature | Integer     |                 | free time after school (numeric: from 1 - very low to 5 - very high)                                                                                   |       | no             |
| goout         | Feature | Integer     |                 | going out with friends (numeric: from 1 - very low to 5 - very high)                                                                                   |       | no             |
| Dalc          | Feature | Integer     |                 | workday alcohol consumption (numeric: from 1 - very low to 5 - very high)                                                                              |       | no             |
| Walc          | Feature | Integer     |                 | weekend alcohol consumption (numeric: from 1 - very low to 5 - very high)                                                                              |       | no             |
| health        | Feature | Integer     |                 | current health status (numeric: from 1 - very bad to 5 - very good)                                                                                    |       | no             |
| absences      | Feature | Integer     |                 | number of school absences (numeric: from 0 to 93)                                                                                                      |       | no             |
| G1            | Target  | Categorical |                 | first period grade (numeric: from 0 to 20)                                                                                                             |       | no             |
| G2            | Target  | Categorical |                 | second period grade (numeric: from 0 to 20)                                                                                                            |       | no             |
| G3            | Target  | Integer     |                 | final grade (numeric: from 0 to 20, output target)                                                                                                     |       | no             |
